In [1]:
import requests
from bs4 import BeautifulSoup
import os
from time import sleep
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
# Function to download a file from a URL
def download_file(url, folder_path, session):
    local_filename = url.split('/')[-1]
    local_filepath = os.path.join(folder_path, local_filename)
    
    # Send GET request to fetch the file content
    with session.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()
        with open(local_filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filepath

In [3]:
# URL of the webpage containing PDF links
url = "https://www.esmats.eu/esmatspapers/completelist.php?whichYear=2023"

# Headers to emulate a request from a Chrome browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

# Create a session with retry strategy
retry_strategy = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    method_whitelist=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session = requests.Session()
session.mount("https://", adapter)
session.mount("http://", adapter)

# Fetch the webpage content
response = session.get(url, headers=headers)
response.raise_for_status()  # Ensure we notice bad responses

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all anchor tags with href attribute ending with .pdf
pdf_links = soup.find_all('a', href=lambda href: href and href.endswith('.pdf'))

# Directory where PDFs will be saved
pdf_folder = 'downloaded_pdfs'
os.makedirs(pdf_folder, exist_ok=True)

# Iterate over all found PDF links and download each PDF
for link in pdf_links:
    pdf_url = link['href']
    # Full URL if the link is relative
    if not pdf_url.startswith('http'):
        pdf_url = 'https://www.esmats.eu' + pdf_url
    print(f'Downloading {pdf_url}')
    try:
        download_file(pdf_url, pdf_folder, session)
    except requests.exceptions.RequestException as e:
        print(f'Failed to download {pdf_url}: {e}')

print('Download completed!')

TypeError: Retry.__init__() got an unexpected keyword argument 'method_whitelist'